In [1]:
!git clone https://github.com/NicolasCambon/RNN_UniversalDependencies_FR.git #Téléchargment des données

Cloning into 'RNN_UniversalDependencies_FR'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 34 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [60]:
import numpy as np

def train(fic):
  print("lecture des donnees d'entrée depuis le fichier : ", fic) 
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]

  POP=[]
  Out=[]
  for i in range(len(sentence)):  
    pa=[]
    no=[]
    if i <= len(sentence)//2:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'

        if "Sing" in sentence[i][e][5]:
          nomb+="_S"
        elif "Plur" in sentence[i][e][5]:
          nomb+="_P"
        pa.append(sentence[i][e][3]+inf+nomb)
        no.append("1")
      POP.append(pa)
      Out.append(no)
      
    else:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'
            
        if sentence[i][e][3] != "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_S"
          elif "Plur" in sentence[i][e][5]:
            nomb+="_P"
          pa.append(sentence[i][e][3]+inf+nomb)
          no.append("1")
        elif sentence[i][e][3] == "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_P"
            break
          elif "Plur" in sentence[i][e][5]:
            nomb+="_S"
            break
          pa.append(sentence[i][e][3]+inf+nomb)
          no.append("0")
          
      POP.append(pa)
      Out.append(no)
  return(POP,Out)

x_train,y_train= train('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt')

lecture des donnees d'entrée depuis le fichier :  RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt


In [61]:
print(x_train[1])
print(y_train[1])

['DET_S', 'NOUN_S', 'PRON_P', 'VERB_S', 'SCONJ', 'ADP', 'DET_S', 'ADJ', 'NOUN_S', 'PUNCT', 'ADP', 'PROPN_S', 'PUNCT', 'ADP', 'ADJ_P', 'NOUN_P', 'PUNCT', 'DET_S', 'NOUN_S', 'ADJ_S', 'VERB_S', 'PUNCT']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']


In [0]:
def distance_verbe(fic):
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]
  num=0
  deno=0
  for i in range (len(sentence)):
    lis=[]
    for j in range(len(sentence[i])):
      if len(sentence[i][j])>8 and sentence[i][j][7]== 'root':
        chiffre = sentence[i][j][0]
        for k in range(len(sentence[i])):
          if len(sentence[i][k])>8 and sentence[i][k][6]== chiffre and 'nsubj'in sentence[i][k][7] and sentence[i][k][0] not in lis:
            lis.append(sentence[i][k][0])
            num += int(chiffre)-int(sentence[i][k][0])
            deno+=1
  return (num/deno)

distance_tr = distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt')
distance_te = distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt')
         

In [49]:
distance

4.569131832797428